In [1]:
import pandas as pd
from pathlib import Path

YEAR = 2020
BASE = Path(f"Hospitales_raw/Hospitales_{YEAR}")


In [ ]:
import pandas as pd
from pathlib import Path

BASE_2020 = Path(".")  # carpeta actual (Hospitales_2020)

print("CWD:", Path.cwd())
print("Archivos:", [p.name for p in BASE_2020.iterdir()])

cat_path  = next(BASE_2020.glob("*Catalogo*.csv"))
ccaa_path = next(BASE_2020.glob("*CCAA*.csv"))
pat_path  = next(BASE_2020.glob("*Patrimonial*.csv"))


print("Usando:", cat_path.name, ccaa_path.name, pat_path.name)


cat  = pd.read_csv(cat_path,  sep=";", encoding="latin1", low_memory=False)
ccaa = pd.read_csv(ccaa_path, sep=";", encoding="latin1", low_memory=False)
pat  = pd.read_csv(pat_path,  sep=";", encoding="latin1", low_memory=False)


# limpiar nombres de columnas (espacios raros)
cat.columns  = cat.columns.str.strip()
ccaa.columns = ccaa.columns.str.strip()
pat.columns  = pat.columns.str.strip()


print("shapes:", cat.shape, ccaa.shape, pat.shape,)
print("CAT cols clave:", [c for c in ["CODAUTO","CODPAT","CODFU","CODPROV"] if c in cat.columns])
print("CCAA cols:", ccaa.columns.tolist())
print("PAT cols:", pat.columns.tolist())


cat.head(3)



CWD: c:\Users\nuez_\Desktop\Proyecto ENSE\Hospitales_raw\Hospitales_2020
Archivos: ['05_Hospitales2020.ipynb', 'CNH_Catalogo_2020.csv', 'CNH_CCAA_2020.csv', 'CNH_Patrimonial_2020.csv']
Usando: CNH_Catalogo_2020.csv CNH_CCAA_2020.csv CNH_Patrimonial_2020.csv
shapes: (837, 24) (19, 3) (11, 2)
CAT cols clave: []
CCAA cols: ['CODAUTO', 'AUTONOMIA', 'TEXTO_AUTONOMIA']
PAT cols: ['COD. DEP. FUNCIONAL', 'DEPENDENCIA FUNCIONAL']


,CCN,CODCNH,Nombre Centro,Tipo Vía,Nombre Vía,Número Vía,Teléfono Principal,Cód. Municipio,Municipio,Cód. Provincia,...,CAMAS,Cód. Clase de Centro,Clase de Centro,Cód. Dep. Funcional,Dependencia Funcional,Forma parte Complejo,CODIDCOM,Nombre del Complejo,ALTA,Email
0,1601000005,10066,HOSPITAL DE CUIDADOS SAN ONOFRE SL,Calle,Salvatierrabide,9,674342402,10590,Vitoria-Gasteiz,1,...,82,C13,Hospitales de media y larga estancia,20,Privados,N,NaN,NaN,N,NaN
1,1601000006,10088,HOSPITAL DE LEZA,Carretera,Vitoria-Logroño Km. 59 ( Er/Cr ),S/N,945006900,10318,Laguardia,1,...,63,C13,Hospitales de media y larga estancia,2,Servicios o Institutos de Salud de las comunid...,N,NaN,NaN,N,NaN
2,1601000004,10053,HOSPITAL QUIRONSALUD VITORIA,Calle,Esperanza (La),3,945252500,10590,Vitoria-Gasteiz,1,...,26,C11,Hospitales Generales,20,Privados,N,NaN,NaN,N,NaN


In [6]:
cat.columns.tolist()

['CCN',
 'CODCNH',
 'Nombre Centro',
 'Tipo Vía',
 'Nombre Vía',
 'Número Vía',
 'Teléfono Principal',
 'Cód. Municipio',
 'Municipio',
 'Cód. Provincia',
 'Provincia',
 'Cód. CCAA',
 'CCAA',
 'Código Postal',
 'CAMAS',
 'Cód. Clase de Centro',
 'Clase de Centro',
 'Cód. Dep. Funcional',
 'Dependencia Funcional',
 'Forma parte Complejo',
 'CODIDCOM',
 'Nombre del Complejo',
 'ALTA',
 'Email']

In [7]:
cat = cat.rename(columns={
    "Cód. CCAA": "CODAUTO",
    "CCAA": "CCAA_nombre",
    "Cód. Dep. Funcional": "COD_DEP_FUNCIONAL",
    "Dependencia Funcional": "DEP_FUNCIONAL"
})





In [8]:
cat["CODAUTO"] = pd.to_numeric(cat["CODAUTO"], errors="coerce").astype("Int64")
cat["COD_DEP_FUNCIONAL"] = pd.to_numeric(cat["COD_DEP_FUNCIONAL"], errors="coerce").astype("Int64")

# En ccaa (tabla diccionario)
ccaa["CODAUTO"] = pd.to_numeric(ccaa["CODAUTO"], errors="coerce").astype("Int64")

# En pat (dependencia funcional)
pat = pat.rename(columns={
    "COD. DEP. FUNCIONAL": "COD_DEP_FUNCIONAL",
    "DEPENDENCIA FUNCIONAL": "DEP_FUNCIONAL_dic"
})
pat["COD_DEP_FUNCIONAL"] = pd.to_numeric(pat["COD_DEP_FUNCIONAL"], errors="coerce").astype("Int64")




In [9]:
base_2020 = cat.merge(
    ccaa[["CODAUTO", "AUTONOMIA"]].rename(columns={"AUTONOMIA": "CCAA"}),
    on="CODAUTO",
    how="left"
)

base_2020["CCAA"].isna().sum(), len(base_2020)


(np.int64(0), 837)

In [10]:
base_2020 = base_2020.merge(
    pat[["COD_DEP_FUNCIONAL", "DEP_FUNCIONAL_dic"]],
    on="COD_DEP_FUNCIONAL",
    how="left"
)

# si existe diccionario, úsalo; si no, usa el texto del catálogo
base_2020["Dependencia_funcional"] = base_2020["DEP_FUNCIONAL_dic"].fillna(base_2020["DEP_FUNCIONAL"])


In [11]:
base_2020

,CCN,CODCNH,Nombre Centro,Tipo Vía,Nombre Vía,Número Vía,Teléfono Principal,Cód. Municipio,Municipio,Cód. Provincia,...,COD_DEP_FUNCIONAL,DEP_FUNCIONAL,Forma parte Complejo,CODIDCOM,Nombre del Complejo,ALTA,Email,CCAA,DEP_FUNCIONAL_dic,Dependencia_funcional
0,1601000005,10066,HOSPITAL DE CUIDADOS SAN ONOFRE SL,Calle,Salvatierrabide,9,674342402,10590,Vitoria-Gasteiz,1,...,20,Privados,N,NaN,NaN,N,NaN,PAÍS VASCO,PRIVADOS,PRIVADOS
1,1601000006,10088,HOSPITAL DE LEZA,Carretera,Vitoria-Logroño Km. 59 ( Er/Cr ),S/N,945006900,10318,Laguardia,1,...,2,Servicios o Institutos de Salud de las comunid...,N,NaN,NaN,N,NaN,PAÍS VASCO,SERVICIOS E INSTITUTOS DE SALUD DE LAS COMUNID...,SERVICIOS E INSTITUTOS DE SALUD DE LAS COMUNID...
2,1601000004,10053,HOSPITAL QUIRONSALUD VITORIA,Calle,Esperanza (La),3,945252500,10590,Vitoria-Gasteiz,1,...,20,Privados,N,NaN,NaN,N,NaN,PAÍS VASCO,PRIVADOS,PRIVADOS
3,111002394,110282,HOSPITAL QUIRON CAMPO DE GIBRALTAR,Calle,Arttysur - P. E. T. Las Marismas De Palmones,S/N,956798301,110086,"Barrios, Los",11,...,20,Privados,N,NaN,NaN,N,NaN,ANDALUCÍA,PRIVADOS,PRIVADOS
4,114000529,140116,HOSPITAL INFANTA MARGARITA,Avenida,Gongora,S/N,957021300,140137,Cabra,14,...,2,Servicios o Institutos de Salud de las comunid...,N,NaN,NaN,N,NaN,ANDALUCÍA,SERVICIOS E INSTITUTOS DE SALUD DE LAS COMUNID...,SERVICIOS E INSTITUTOS DE SALUD DE LAS COMUNID...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
832,1328020933,281438,INSTITUTO PSIQUIATRICO SERVICIOS DE SALUD MENT...,Calle,Aragon,17,000000000,280745,Leganés,28,...,2,Servicios o Institutos de Salud de las comunid...,S,281427.0,Instituto Psiquiátrico Servicios De Salud Ment...,N,NaN,MADRID,SERVICIOS E INSTITUTOS DE SALUD DE LAS COMUNID...,SERVICIOS E INSTITUTOS DE SALUD DE LAS COMUNID...
833,1430000036,300362,HOSPITAL GENERAL UNIVERSITARIO SANTA LUCIA,Calle,Mezquita (Paraje Los Arcos),S/N,968128600,300161,Cartagena,30,...,2,Servicios o Institutos de Salud de las comunid...,S,300351.0,Complejo Hospitalario De Cartagena Chc,N,NaN,REGIÓN DE MURCIA,SERVICIOS E INSTITUTOS DE SALUD DE LAS COMUNID...,SERVICIOS E INSTITUTOS DE SALUD DE LAS COMUNID...
834,1430000062,300145,HOSPITAL SANTA MARIA DEL ROSELL,Paseo,Alfonso Xiii,61,968504800,300161,Cartagena,30,...,2,Servicios o Institutos de Salud de las comunid...,S,300351.0,Complejo Hospitalario De Cartagena Chc,N,NaN,REGIÓN DE MURCIA,SERVICIOS E INSTITUTOS DE SALUD DE LAS COMUNID...,SERVICIOS E INSTITUTOS DE SALUD DE LAS COMUNID...
835,1726000067,260048,HOSPITAL DE LA RIOJA,Avenida,Avenida De Viana,1,941298000,260892,Logroño,26,...,2,Servicios o Institutos de Salud de las comunid...,S,260012.0,Complejo Hospitalario San Pedro Hospital De La...,N,gerencia_srs@riojasalud.es,LA RIOJA,SERVICIOS E INSTITUTOS DE SALUD DE LAS COMUNID...,SERVICIOS E INSTITUTOS DE SALUD DE LAS COMUNID...


In [12]:
base_2020["año"] = 2020


In [13]:
dep = base_2020["Dependencia_funcional"].astype(str).str.upper()

base_2020["titularidad"] = "Otros"
base_2020.loc[dep.str.contains("PRIV"), "titularidad"] = "Privado"
base_2020.loc[dep.str.contains("COMUNIDAD|SEGURIDAD SOCIAL|MINISTERIO|DIPUTACIÓN|CABILDO|MUNICIPIO|ENTIDAD"), "titularidad"] = "Público"

base_2020["titularidad"].value_counts(dropna=False)



titularidad
Privado    421
Público    382
Otros       34
Name: count, dtype: int64

In [14]:
agg_2020 = (
    base_2020
    .groupby(["CCAA", "año", "titularidad"], dropna=False)
    .size().reset_index(name="n_hospitales")
)

tot_2020 = (
    base_2020
    .groupby(["CCAA", "año"], dropna=False)
    .size().reset_index(name="total_hospitales")
)

agg_2020 = agg_2020.merge(tot_2020, on=["CCAA", "año"], how="left")
agg_2020["pct"] = (agg_2020["n_hospitales"] / agg_2020["total_hospitales"]) * 100

agg_2020.head(10)



,CCAA,año,titularidad,n_hospitales,total_hospitales,pct
0,ANDALUCÍA,2020,Otros,1,127,0.787402
1,ANDALUCÍA,2020,Privado,59,127,46.456693
2,ANDALUCÍA,2020,Público,67,127,52.755906
3,ARAGÓN,2020,Otros,4,29,13.793103
4,ARAGÓN,2020,Privado,6,29,20.689655
5,ARAGÓN,2020,Público,19,29,65.517241
6,C. FORAL DE NAVARRA,2020,Privado,6,10,60.000000
7,C. FORAL DE NAVARRA,2020,Público,4,10,40.000000
8,CANARIAS,2020,Otros,2,39,5.128205
9,CANARIAS,2020,Privado,22,39,56.410256


In [16]:
from pathlib import Path

# si tu DF se llama distinto, cambia agg_2020 por tu variable real
df_out = agg_2020.copy()

# carpeta Data_clean (desde tu estructura del proyecto)
out_path = Path("../../Data_clean") / "Hospitales_agregado_2020.csv"

df_out.to_csv(out_path, index=False, encoding="utf-8-sig")
out_path



WindowsPath('../../Data_clean/Hospitales_agregado_2020.csv')